Ce notebook va servir à la création de notre vecteur Y et notre tableau X de 2006 à 2022 sur la saison estivale afin d'appliquer ensuite nos modèles de prédiction.

In [4]:
import pandas as pd

## Création du vecteur Y ##

Y est un vecteur de 0 et de 1. Chaque ligne correspond à une commune, qui vont être répétées de 2006 à 2022 :
- $Y_1$ = 1 si il y a eu au moins un feu dans la commune 1 sur la période estivale en 2006, 0 sinon
- $Y_2$ = 1 si il y a eu au moins un feu dans la commune 2 sur la période estivale en 2006, 0 sinon
  ...

In [5]:
data = 'combine.csv'

df = pd.read_csv('combine.csv', sep=';')
#print(df.head())

# Filtrer les lignes où la valeur de la colonne "Département" est égale à 40
bdiff_landes = df[df['Département'] == '40']

# Afficher les premières lignes du DataFrame filtré
print(bdiff_landes.head())

     Année  Numéro Département Code INSEE  Nom de la commune  \
652   2006    1876          40      40006          Arengosse   
653   2006    1877          40      40151             Lesgor   
654   2006    1878          40      40108             Gastes   
655   2006    1879          40      40210  Onesse-et-Laharie   
656   2006    1881          40      40167              Luxey   

    Date de première alerte Origine de l'alerte  \
652        26/07/2006 19:40         Indéterminé   
653        17/07/2006 15:29         Indéterminé   
654        25/04/2006 18:40         Indéterminé   
655        17/07/2006 18:20         Indéterminé   
656        27/07/2006 10:49         Indéterminé   

     Moyens de première intervention  Surface parcourue (m2)  \
652                              NaN                    5000   
653                              NaN                   66000   
654                              NaN                     400   
655                              NaN                

*Test uniquement pour l'année 2006*

In [9]:
# Conversion du format de la date
bdiff_landes.loc[:, 'Date de première alerte'] = pd.to_datetime(bdiff_landes['Date de première alerte'], format='%d/%m/%Y %H:%M')

# Filtrer pour l'année 2006
bdiff_landes_2006 = bdiff_landes[bdiff_landes['Année'] == 2006]

# Liste complète des communes en 2006
communes = bdiff_landes_2006['Code INSEE'].drop_duplicates().to_frame()

# Filtrer pour les mois de juin, juillet et août
df_summer = bdiff_landes_2006[bdiff_landes_2006['Date de première alerte'].dt.month.isin([6, 7, 8])]

# Aggrégation par commune avec vérification des incendies
feu_par_commune = df_summer.groupby('Code INSEE').size().gt(0).reset_index(name='Feu en été')

# Convertir les valeurs en 1 et 0
feu_par_commune['Feu en été'] = feu_par_commune['Feu en été'].astype(int)

# Jointure avec la liste complète des communes pour inclure les communes sans feu
resultat_final = communes.merge(feu_par_commune, on='Code INSEE', how='left')
resultat_final['Feu en été'] = resultat_final['Feu en été'].fillna(0).astype(int)

# Afficher le résultat
print(resultat_final)


   Code INSEE  Feu en été
0       40006           1
1       40151           1
2       40108           0
3       40210           1
4       40167           1
..        ...         ...
83      40329           0
84      40143           1
85      40258           0
86      40015           0
87      40030           0

[88 rows x 2 columns]


On a pas toutes les communes, juste celles qui apparaissent dans combine en 2006. Il faut y ajouter celles qui n'apparaissent pas (car pas de feux dans l'année)

In [ ]:
# communes_landes = pd.read_csv('chemin_vers_liste_des_communes.csv')

# Convertissez la colonne de date si nécessaire
df['Date de première alerte'] = pd.to_datetime(df['Date de première alerte'], format='%d/%m/%Y %H:%M')

# Filtrer pour l'année 2006 et les mois d'été
df_summer = df[(df['Année'] == 2006) & (df['Date de première alerte'].dt.month.isin([6, 7, 8]))]

# Groupement par commune avec vérification des feux
feu_par_commune = df_summer.groupby('Code INSEE').size().gt(0).reset_index(name='Feu en été')
feu_par_commune['Feu en été'] = feu_par_commune['Feu en été'].astype(int)

# Fusionner avec la liste complète des communes
resultat_final = communes_landes.merge(feu_par_commune, on='Code INSEE', how='left')
resultat_final['Feu en été'] = resultat_final['Feu en été'].fillna(0).astype(int)

# Afficher le résultat
print(resultat_final)
